In [1]:
import tensorflow as tf
import numpy as np
from common import *

from asyncio.events import  AbstractEventLoop
import logging
import os, time
from numba import jit
import tflearn
from tflearn.layers.core import time_distributed
from tflearn.layers import conv_2d
from io import StringIO

logging.basicConfig(level=logging.DEBUG)
os.environ['PYTHONASYNCIODEBUG'] = '1'

In [210]:
def parse_csv(text):
    strings = tf.string_split([text], delimiter='\n')
    raw_nums = tf.string_split(strings.values)
    nums = tf.string_to_number(raw_nums.values, tf.int32)

    dense = tf.sparse_to_dense(raw_nums.indices, 
                               raw_nums.dense_shape, 
                               nums,
                               default_value=0)
#     dense.set_shape(raw_nums.get_shape())
    return dense


def read_input_file(filename_queue):
    reader = tf.WholeFileReader()
    key, record_string = reader.read(filename_queue)
    example = parse_csv(record_string)
#     processed_example = some_processing(example)
    return example, key


def input_pipeline(filenames, batch_size, num_epochs=None):
    filename_queue = tf.train.string_input_producer(
      filenames, num_epochs=num_epochs, shuffle=False, seed=0)
    example, key = read_input_file(filename_queue)    

    min_after_dequeue = 10000
    capacity = min_after_dequeue + 3 * batch_size
    example_batch, label_batch = tf.train.batch(
        [example, key], 
        batch_size=batch_size, 
        capacity=capacity,
#         min_after_dequeue=min_after_dequeue,
        dynamic_pad=True,
        allow_smaller_final_batch=True,
        num_threads=cpu_count
    )
    return example_batch, label_batch

In [208]:
filenames = glob('../data/corpus/*.txt')[100:112]
filenames = np.reshape(filenames, (-1,3))

In [209]:
filenames

array([['../data/corpus/5984bf58b6b11307a2638529.txt',
        '../data/corpus/5984c732b6b113644f638532.txt',
        '../data/corpus/5984bd1eb6b1136c756384f5.txt'],
       ['../data/corpus/5984c1fdb6b1132b6863853e.txt',
        '../data/corpus/5984cfd2b6b113399a6384f7.txt',
        '../data/corpus/5984c4eeb6b1134ab8638530.txt'],
       ['../data/corpus/5984cb0ab6b1130b9863852c.txt',
        '../data/corpus/5984d818b6b1130a266384f1.txt',
        '../data/corpus/5984dbd1b6b113237863853f.txt'],
       ['../data/corpus/5984c861b6b1136d74638511.txt',
        '../data/corpus/5984c20db6b1132c8e6384f7.txt',
        '../data/corpus/5984bb6db6b11359be638527.txt']],
      dtype='<U43')

In [211]:
start_time = time.time()

g = tf.Graph()
with g.as_default():  
    tf.set_random_seed(0)
    sess = tf.Session()
    with sess.as_default():
        example_batch, label_batch = input_pipeline(filenames, batch_size=2, num_epochs=1)        
        
        init_local = tf.local_variables_initializer()
        init_global = tf.global_variables_initializer()
        sess.run([init_global, init_local])
                
        # Start input enqueue threads.
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)

        try:
            while not coord.should_stop():
                batches, keys = sess.run([example_batch, label_batch])  
#                 time_distributed(batches, conv_2d, [num_filters, filter_sizes, strides])
                pprint(batches.shape)
        except tf.errors.OutOfRangeError:
            print('Done training -- epoch limit reached')
        finally:
            # When done, ask the threads to stop.
            coord.request_stop()

        # Wait for threads to finish.
        coord.join(threads)        

print("--- %s seconds ---" % (time.time() - start_time))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

# My input func

In [166]:
num_threads=cpu_count

def input_pipeline(filenames, batch_size, num_epochs=None):    
    filename_queue = tf.train.string_input_producer(
      filenames, num_epochs=num_epochs, shuffle=False, seed=0)
    example_queue, key = read_input_file(filename_queue)    

    # Create a training graph that starts by dequeuing a batch of examples.
    inputs = example_queue.dequeue_up_to(batch_size)
#     train_op = ...use 'inputs' to build the training part of the graph...

    
#     min_after_dequeue = 10000
#     capacity = min_after_dequeue + 3 * batch_size
#     example_batch, label_batch = tf.train.batch(
#         [example, key], 
#         batch_size=batch_size, 
#         capacity=capacity,
# #         min_after_dequeue=min_after_dequeue,
#         dynamic_pad=True,
#         allow_smaller_final_batch=True,
#         num_threads=cpu_count
#     )
#     return example_batch, label_batch
    return inputs


g = tf.Graph()
with g.as_default():  
    tf.set_random_seed(0)
    sess = tf.Session()
    with sess.as_default():
        inputs = input_pipeline(filenames, batch_size=2, num_epochs=1)        
        
        init_local = tf.local_variables_initializer()
        init_global = tf.global_variables_initializer()
        sess.run([init_global, init_local])
                
        # Start input enqueue threads.
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)

        try:
            while not coord.should_stop():
                examples = sess.run([inputs, label_batch])  
#                 time_distributed(batches, conv_2d, [num_filters, filter_sizes, strides])
                pprint(keys)
        except tf.errors.OutOfRangeError:
            print('Done training -- epoch limit reached')
        finally:
            # When done, ask the threads to stop.
            coord.request_stop()

        # Wait for threads to finish.
        coord.join(threads)        

AttributeError: 'Tensor' object has no attribute 'dequeue_up_to'